# Packages

In [1]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra, StatsBase, HDF5, Random
cd(desktopPath) # change directory

# Initial setup

In [2]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 15
tfinal = 140 * dt
nsteps = round(tfinal / dt) + 1
println("Step takes $(sciNotation(dt, 3)) s")
println("Total of $(round(Int, nsteps)) steps")
println("Final time: $(sciNotation(tfinal, 3)) s")

Step takes 1.333E-4 s
Total of 141 steps
Final time: 1.867E-2 s


# Mesh

Geometry

In [3]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 10, 26)) # 1
push!(Meshes, Q4quadrilateral([0.0 2.6; 1.6 5.45], 13, 30)) # 2
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 25, 24)) # 3
push!(Meshes, Q4quadrilateral([1.6 2.6; 4.05 5.8], 23, 34)) # 4
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 4.5], 10, 35)) # 5
push!(Meshes, Q4quadrilateral([4.05 4.5; 5.25 5.3], 12, 9)) # 6
push!(Meshes, Q4quadrilateral([4.25 5.3; 5.25 5.85], 10, 4)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 7.0 3.92], 22, 29)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.05)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))
println("$(size(fens.xyz, 1)) nodes")
println("$(size(fes.conn, 1)) elements")

3362 nodes
3168 elements


Subdomain selection

In [4]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected)
fess = renumberconn!(fes, new_numbering)
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1));

Export domain

In [5]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
# vtkexportmesh("./vtkFiles/interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4);

# Discrete model

Setup

In [6]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm  =  FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S  =  acousticstiffness(femm, geom, P)
C  =  acousticmass(femm, geom, P)
# "damping" matrix for the absorbing boundary conditions (ABCs)
D  =  acousticABC(
  FEMMAcoustSurf(IntegDomain(bfes, GaussRule(2, 2)), material), geom, P
)
# node IDs for load positions
loadNode = eachindex(fens) |> collect
# initial condition as load in nodes of an element
function pointLoad(dpdn, xyz, J, label, t)::Float64
  dpdn[1] = t == dt ? -rho * 0.02 : 0.0
end;

Point selection

In [7]:
ICamount = 1000 # amound of initial conditions
# IDs of load elements (different initial conditions)
loadID = (collect(1 : size(fes.conn, 1)) |> shuffle)[1 : ICamount]
@show size(loadID)
# sensors are chosen as the first node of each load element.
# therefore: number of sensors = number of ICs

# random points in random instants.
# points y for evaluating solution G(u)(y).
# inputs to trunk network.
# matrix with rows [nodeID x y t]
yAmount = 1000
# IDs of chosen nodes
yNodeID = (collect(1 : size(fens.xyz, 1)) |> shuffle)[1 : yAmount]
y = hcat(
  yNodeID,
  fens.xyz[yNodeID, :],
  rand(1:nsteps, yAmount)' |> vec
)
@show size(y)
println("HDF5 file contains $(
  4 * yAmount + ICamount * yAmount
) values across $(ICamount * yAmount) samples.")

size(loadID) = (1000,)


size(y) = (1000, 4)
HDF5 file contains 1004000 values across 1000000 samples.


Solution

In [8]:
# dictionary for dataset
data = Dict(
  # spatial-temporal points for evaluating G(u). trunk inputs
  "solEvalPoints" => y,
  # G(u)(y). Targets for DeepONet outputs.
  # each row refers to an IC case; each column, to a point y
  "solsVals" => zeros(Float32, (ICamount, yAmount))
)
for (count, loadElementID) in enumerate(loadID)
  evalPoints = 0
  count % 100 == 0 && println("Element $count/$ICamount")
  # element to be loaded in current IC
  dipfemm = FEMMAcoustSurf(IntegDomain(subset(fes, [loadElementID]), GaussRule(2, 2)), material)
  t = 0.0 # Initial time
  # Solve the transient acoustics equations.
  # The loop executes inside the "let" local scope
  P1 = let
    P0 = deepcopy(P)
    P0.values .= 0.0 # initially all pressure is zero
    vP0 = gathersysvec(P0)
    vQ0 = zeros(eltype(vP0), size(vP0))
    # The `P1` field will be the output of this computation:
    # the final value of the pressure field
    P1 = deepcopy(P0)
    fi = ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    La0 = distribloads(dipfemm, geom, P1, fi, 2)
    A = (2.0 / dt) * S + D + (dt / 2.0) * C
    step = 0
    while t <= tfinal
      step += 1
      t += dt
      # Update load
      fi = ForceIntensity( # Initial load
        FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
      )
      La1 = distribloads(dipfemm, geom, P1, fi, 2)
      # Solve for the rate of the pressure
      vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
      # Update the value of the pressure
      vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
      # Swap variables for next step  
      vP0 = deepcopy(vP1); vQ0 = deepcopy(vQ1)
      P1 = scattersysvec!(P1, vec(vP1))
      P0 = deepcopy(P1); La0 = deepcopy(La1)
      # find rows in data["solEvalPoints"] that refer to current timestep
      yRows = findall(k -> k[4] == step, eachrow(data["solEvalPoints"]))
      evalPoints += length(yRows)
      # read pressure values in nodes of interest in current timestep, and
      # use those values to populate the row of data["solsVals"]
      # referring to the current IC case
      data["solsVals"][count, yRows] .= (real.(P1.values) |> vec)[
        Int.(data["solEvalPoints"][yRows, 1])
      ]
    end
    P1
  end
end

Element 100/1000


Element 200/1000


Element 300/1000


Element 400/1000


Element 500/1000


Element 600/1000


Element 700/1000


Element 800/1000


Element 900/1000


Element 1000/1000


# Data

Normalize

In [17]:
standardData = copy(data)
unitData = copy(data)
# standardize
standardData["solEvalPoints"] = Float32.(deepcopy(data["solEvalPoints"][:, 2:4]))
standardData["solEvalPoints"] = (
  standardData["solEvalPoints"] .- mean(standardData["solEvalPoints"])
) ./ std(standardData["solEvalPoints"])
standardData["solsVals"] = (data["solsVals"] .- mean(data["solsVals"])) ./ std(data["solsVals"])
# unit normalization
unitData["solEvalPoints"] = Float32.(deepcopy(data["solEvalPoints"][:, 2:4]))
unitData["solEvalPoints"] .-= minimum(unitData["solEvalPoints"])
unitData["solEvalPoints"] ./= maximum(unitData["solEvalPoints"])
unitData["solsVals"] = data["solsVals"] .- minimum(data["solsVals"])
unitData["solsVals"] ./= maximum(unitData["solsVals"]);

1000×1000 Matrix{Float32}:
 0.923835  0.925561  0.871387  0.925561  …  0.923087  0.925561  0.878096
 0.915112  0.903553  0.925561  0.890351     0.908966  0.91292   0.925561
 0.927692  0.925561  0.925562  0.925561     0.922434  0.925561  0.872682
 0.906736  0.925867  0.925561  0.923449     0.916719  0.929066  0.925561
 0.902041  0.890938  0.925561  0.925052     0.908786  0.904365  0.925561
 0.921846  0.925561  0.851825  0.925561  …  0.922842  0.925561  0.89003
 0.905627  0.904567  0.925561  0.907457     0.907542  0.917387  0.925561
 0.902242  0.892817  0.925561  0.925712     0.900231  0.913861  0.925561
 0.923642  0.925561  0.862561  0.925561     0.923428  0.925561  0.879939
 0.923767  0.925561  0.862491  0.925561     0.923171  0.925561  0.893707
 ⋮                                       ⋱                      
 0.911332  0.925561  0.925561  0.925564     0.920057  0.888938  0.925561
 0.921631  0.925561  0.773514  0.925561     0.923173  0.925561  0.882361
 0.90223   0.896031  0.925561  0.

Save in HDF5

In [18]:
h5open("./vtkFiles/DeepONetData.h5", "w") do dataFile
  dataFile["solEvalPoints"] = data["solEvalPoints"][:, 2:4]
  dataFile["solsVals"] = data["solsVals"]
  dataFile["standardSolEvalPoints"] = standardData["solEvalPoints"]
  dataFile["standardSolsVals"] = standardData["solsVals"]
  dataFile["unitSolEvalPoints"] = unitData["solEvalPoints"]
  dataFile["unitSolsVals"] = unitData["solsVals"]
  dataFile["nodeCoords"] = Float32.(fens.xyz)
  dataFile["nodeConnectivity"] = Base.Iterators.flatten(fes.conn) |> collect
  dataFile["loadNodeIDs"] = loadID
end;

Summary statistics of file

In [19]:
h5open("./vtkFiles/DeepONetData/DeepONetData.h5", "r") do f
  for field in HDF5.get_datasets(f)
    println("\n", HDF5.name(field)[2 : end])
    d = HDF5.read(field)
    @show size(d)
    statsum(d)
  end
end


loadNodeIDs
size(d) = (1000,)
Summary Stats:
Length:         1000
Missing Count:  0
Mean:           1611.209000
Minimum:        2.000000
1st Quartile:   818.500000
Median:         1603.500000
3rd Quartile:   2413.250000
Maximum:        3167.000000
Standard deviation: 9.2142E2
1000/1000 (100.0%) non-zero elements.


nodeConnectivity
size(d) = (12672,)
Summary Stats:
Length:         12672
Missing Count:  0
Mean:           1685.899858
Minimum:        1.000000
1st Quartile:   864.000000
Median:         1681.500000
3rd Quartile:   2516.250000
Maximum:        3362.000000
Standard deviation: 9.6353E2
12672/12672 (100.0%) non-zero elements.


nodeCoords
size(d) = (3362, 2)
Summary Stats:
Length:         6724
Missing Count:  0
Mean:           3.136134
Minimum:        0.000000
1st Quartile:   1.733333
Median:         3.075862
3rd Quartile:   4.500000
Maximum:        7.000000
Standard deviation: 1.76E0
6630/6724 (98.6%) non-zero elements.


solEvalPoints
size(d) = (1000, 3)
Summary Stats:
Length


Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           -0.000000
Minimum:        -0.000012
1st Quartile:   -0.000000
Median:         -0.000000
3rd Quartile:   0.000000
Maximum:        0.000001
Standard deviation: 3.0364E-7
940712/1000000 (94.1%) non-zero elements.


standardSolEvalPoints
size(d) = (1000, 3)
Summary Stats:
Length:         3000
Missing Count:  0
Mean:           0.000000
Minimum:        -0.647015
1st Quartile:   -0.585003
Median:         -0.537010
3rd Quartile:   0.187502
Maximum:        2.918649
Standard deviation: 1.0E0
3000/3000 (100.0%) non-zero elements.


standardSolsVals
size(d) = (1000, 1000)


Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           0.000000
Minimum:        -37.797806
1st Quartile:   -0.257113
Median:         0.476192
3rd Quartile:   0.476195
Maximum:        3.554416
Standard deviation: 10.0E-1
1000000/1000000 (100.0%) non-zero elements.


unitSolEvalPoints
size(d) = (1000, 3)
Summary Stats:
Length:         3000
Missing Count:  0
Mean:           0.181457
Minimum:        0.000000
1st Quartile:   0.017391
Median:         0.030851
3rd Quartile:   0.234043
Maximum:        1.000000
Standard deviation: 2.8045E-1
2976/3000 (99.2%) non-zero elements.


unitSolsVals
size(d) = (1000, 1000)


Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           0.914045
Minimum:        0.000000
1st Quartile:   0.907828
Median:         0.925561
3rd Quartile:   0.925561
Maximum:        1.000000
Standard deviation: 2.4182E-2
999998/1000000 (100.0%) non-zero elements.

